In [612]:
import os
import pprint
import shutil
import importlib 

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad


from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.runrecord_overlap as runrecord_overlap



importlib.reload(setup)
importlib.reload(pipeline)
importlib.reload(runrecord_clirad)
importlib.reload(utils_clirad)
importlib.reload(setup_overlap)
importlib.reload(setup_bestfit)
importlib.reload(pipeline_lblnew)
importlib.reload(bestfit)
importlib.reload(runrecord_bestfit)
importlib.reload(runrecord_overlap)

<module 'climatools.lblnew.runrecord_overlap' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/runrecord_overlap.py'>

In [673]:
def params_clirad_nonoverlap():
    '''
    Define clirad-lw input parameter dictionaries
    for 2xco2, co2-only runs.
    '''
    commitnumber = 'c3e1ce5'
    molecule = {'co2': 800e-6}
    bands = [3, 4, 5, 6, 7, 11] # these are the only bands with co2
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    
    params = []
    for band in bands:
        for atmpro, tsfc in atmpros:
            param = {'commitnumber': commitnumber,
                     'molecule': molecule,
                     'band': [band],
                     'atmpro': atmpro,
                     'tsfc': tsfc}
            params.append(param)
    return params



def param_lblnewbestfit():
    '''    
    lblnew-bestfit input parameter dictionaries
    matching those from clirad-lw's 2xco2, 
    co2-only runs.
    '''
    commitnumber = 'a22ab94'
    
    params_clirad = clirad_params()
    record_bestfit = runrecord_bestfit.params()
    
    params = []
    for param_clirad in params_clirad:
        
        molecule = list(param_clirad['molecule'].keys())[0]
        conc = param_clirad['molecule']['co2']
        band = param_clirad['band'][0]
        band = utils_clirad.mapband_new2old()[band]
        atmpro = param_clirad['atmpro']
        tsfc = param_clirad['tsfc']
        
        param_bestfit = [r for r in record_bestfit
                         if r['commitnumber'] == commitnumber
                         if r['molecule'] == molecule
                         if r['conc'] == conc
                         if r['band'] == band
                         if r['atmpro'] == atmpro
                         if r['tsfc'] == tsfc]
        
        assert len(param_bestfit) == 1
        param_bestfit = param_bestfit[0]
        params.append(param_bestfit)
        
    return params


def parampairs_nonoverlap():
    params_clirad = params_clirad_nonoverlap()
    params_lblnew = param_lblnewbestfit_nonoverlap()
    
    assert len(params_clirad) == len(params_lblnew)
    
    return list(zip(params_clirad, params_lblnew))


pprint.pprint(parampairs_nonoverlap())

[({'atmpro': 'mls',
   'band': [3],
   'commitnumber': 'c3e1ce5',
   'molecule': {'co2': 0.0008},
   'tsfc': 294},
  {'atmpro': 'mls',
   'band': '3a',
   'commitnumber': 'a22ab94',
   'conc': 0.0008,
   'dv': 0.001,
   'klin': 6.375563e-24,
   'molecule': 'co2',
   'ng_adju': [0, 0, 0],
   'ng_refs': [3, 2, 4],
   'nv': 1000,
   'option_compute_btable': 0,
   'option_compute_ktable': 0,
   'option_wgt_flux': 1,
   'option_wgt_k': 1,
   'ref_pts': [(1, 250), (10, 250), (500, 250)],
   'tsfc': 294,
   'vmax': 620,
   'vmin': 540,
   'w_diffuse': [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)],
   'wgt': [(0.7, 0.3, 0.7), (0.7, 0.6), (0.4, 0.5, 0.8, 0.95)]}),
 ({'atmpro': 'saw',
   'band': [3],
   'commitnumber': 'c3e1ce5',
   'molecule': {'co2': 0.0008},
   'tsfc': 257},
  {'atmpro': 'saw',
   'band': '3a',
   'commitnumber': 'a22ab94',
   'conc': 0.0008,
   'dv': 0.001,
   'klin': 6.375563e-24,
   'molecule': 'co2',
   'ng_adju': [0, 0, 0],
   'ng_refs': [3, 2, 4],
   'nv': 10

In [ ]:
def params_clirad_overlap():
    '''
    Define clirad-lw input parameter dictionaries
    for 2xco2, co2 overlapping with others.
    '''
    commitnumber = ''
    molecule = {'co2': 800e-6}
    bands = [3, 4, 5, 6,7, 11] # these are the only bands with co2
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    
    params = []
    for band in bands:
        for atmpro, tsfc in atmpros:
            param = {'commitnumber': commitnumber,
                     'molecule': molecule,
                     'band': [band],
                     'atmpro': atmpro,
                     'tsfc': tsfc}
            params.append(param)
    return params



def param_lblnewoverlap():
    '''    
    lblnew-bestfit input parameter dictionaries
    matching those from clirad-lw's 2xco2, 
    co2 overlapping with others.
    '''
    commitnumber = 'a22ab94'
    
    params_clirad = clirad_params()
    record_bestfit = runrecord_bestfit.params()
    
    params = []
    for param_clirad in params_clirad:
        
        molecule = list(param_clirad['molecule'].keys())[0]
        conc = param_clirad['molecule']['co2']
        band = param_clirad['band'][0]
        band = utils_clirad.mapband_new2old()[band]
        atmpro = param_clirad['atmpro']
        tsfc = param_clirad['tsfc']
        
        param_bestfit = [r for r in record_bestfit
                         if r['commitnumber'] == commitnumber
                         if r['molecule'] == molecule
                         if r['conc'] == conc
                         if r['band'] == band
                         if r['atmpro'] == atmpro
                         if r['tsfc'] == tsfc]
        
        assert len(param_bestfit) == 1
        param_bestfit = param_bestfit[0]
        params.append(param_bestfit)
        
    return params

In [280]:
# lblnew-bestfit cases

def param_lblnew(molecule=None, band=None):
    param = bestfit.kdist_params(molecule=molecule, band=band)
    param['option_compute_ktable'] = 0
#    param['commitnumber'] = '5014a19'
#    param['commitnumber'] = 'a06b618'
#    param['commitnumber'] = 'a22ab94'
    return param


def param_cliradlw(molecule=None, band=None):
    
    param = bestfit.kdist_params(molecule=molecule, band=band)

    clirad_band = int(band_map()[band])
    conc = 'atmpro' if param['conc'] == None else param['conc']
    
    return {'atmpro': param['atmpro'],
            'band': [clirad_band],
            'commitnumber': 'de5d43e',
           'molecule': {molecule: conc},
            'tsfc': param['tsfc']}


def parampairs():
    ps = []
    
    for molecule, band in molbands():
        for atmpro, tsfc in [('mls', 294), ('saw', 257), ('trp', 300)]:
            p_lblnew = param_lblnew(molecule=molecule, band=band)
            p_clirad = param_cliradlw(molecule=molecule, band=band)
            p_lblnew['atmpro'] = atmpro
            p_lblnew['tsfc'] = tsfc
            p_clirad['atmpro'] = atmpro
            p_clirad['tsfc'] = tsfc
        
            ps.append((p_clirad, p_lblnew))
    
    return ps

In [667]:
# lblnew-overlap cases


def parampairs():
    ps = []
    
    param = {'commitnumber': 'b42ec27',                                                                                                                                                       
             'molecule': {'h2o': 'atmpro', 'co2': 400e-6, 'n2o': 3.2e-7},                                                                                                                     
             'band': '3a',                                                                                                                                                                    
             'nv': 1000,                                                                                                                                                                      
             'dv': .001,                                                                                                                                                                      
             'atmpro': 'mls',                                                                                                                                                                 
             'tsfc': 294}
    
    for atmpro, tsfc in [('mls', 294), ('saw', 257), ('trp', 300)]:
        param_lblnew = param.copy()
        param_lblnew['atmpro'] = atmpro
        param_lblnew['tsfc'] = tsfc
        
        param_clirad = param.copy()
        param_clirad['commitnumber'] = '9d7b225'
        param_clirad['band'] = [int(band_map()[param['band']])]
        param_clirad['atmpro'] = atmpro
        param_clirad['tsfc'] = tsfc
        del param_clirad['nv']
        del param_clirad['dv']
        
        ps.append((param_clirad, param_lblnew))
    return ps



In [620]:
def script_clirad_lblnew():
    '''
    Script to run clirad and then plot results against the 
    corresponding lblnew case.  This needs both input parameter
    dictionaries from clirad and lblnew.
    '''
    ps = parampairs_nonoverlap()
    
    gprocs = pipeline.pipeline_fortran2ipynb2git(parampairs=ps, 
                                                 setup=setup)    

In [641]:
script_clirad_lblnew()

Submitting radiation calculation for cases
{'atmpro': 'mls',
 'band': [3],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [3],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [3],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [4],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [4],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [4],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [5],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [5],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [5],
 'commitnumber': 'c3e1ce5',
 'molecule': {'co2': 0.0008},
 'tsfc': 300}
{'atmpro': 'mls',
 'b

In [670]:
# No lblnew cases compared. For running clirad-lw only.

def get_params():
    param0 = {'commitnumber': '20cb483',
              'molecule': {'h2o': 'atmpro',
                           'co2': 400e-6,
                           'o3': 'atmpro',
                           'n2o': 3.2e-7,
                           'ch4': 1.8e-6},
              'band': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
              'atmpro': 'mls',
              'tsfc': 294,}

    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]

    params = []
    for atmpro, tsfc in atmpros:
        param = param0.copy() 
        param['atmpro'] = atmpro
        param['tsfc'] = tsfc
        params.append(param)
        
    return params



def script_clirad_only():
    for param in get_params():
        try:
            fdir = pipeline.get_fortran_dir(param=param, setup=setup)
            shutil.rmtree(fdir)
        except FileNotFoundError:
            pass
        
        fproc = pipeline.run_fortran(param=param, setup=setup)

In [671]:
script_clirad_only()

{'atmpro': 'mls',
 'band': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'commitnumber': '20cb483',
 'molecule': {'ch4': 1.8e-06,
              'co2': 0.0004,
              'h2o': 'atmpro',
              'n2o': 3.2e-07,
              'o3': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'commitnumber': '20cb483',
 'molecule': {'ch4': 1.8e-06,
              'co2': 0.0004,
              'h2o': 'atmpro',
              'n2o': 3.2e-07,
              'o3': 'atmpro'},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'commitnumber': '20cb483',
 'molecule': {'ch4': 1.8e-06,
              'co2': 0.0004,
              'h2o': 'atmpro',
              'n2o': 3.2e-07,
              'o3': 'atmpro'},
 'tsfc': 300}


In [672]:
for param in get_params():
    fdir = pipeline.get_fortran_dir(param=param, setup=setup)
    
    print(fdir)
    print(os.listdir(fdir))
    print()

/chia_cluster/home/jackyu/radiation/clirad-lw/LW/examples/lblnew/h2o_mls_co2_0.0004_o3_mls_n2o_3.2e-07_ch4_1.8e-06/band_1_2_3_4_5_6_7_8_9_10_11/atmpro_mls/cliradlw_20cb483
['cliradlw.exe', 'CLIRAD_new_25cm_re.f']

/chia_cluster/home/jackyu/radiation/clirad-lw/LW/examples/lblnew/h2o_saw_co2_0.0004_o3_saw_n2o_3.2e-07_ch4_1.8e-06/band_1_2_3_4_5_6_7_8_9_10_11/atmpro_saw/cliradlw_20cb483
['cliradlw.exe', 'CLIRAD_new_25cm_re.f']

/chia_cluster/home/jackyu/radiation/clirad-lw/LW/examples/lblnew/h2o_trp_co2_0.0004_o3_trp_n2o_3.2e-07_ch4_1.8e-06/band_1_2_3_4_5_6_7_8_9_10_11/atmpro_trp/cliradlw_20cb483
['cliradlw.exe', 'CLIRAD_new_25cm_re.f']

